In [1]:
!pip install transformers
!pip install datasets
!pip install torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn import functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


In [5]:
from google.colab import files
uploaded = files.upload()


Saving pokemon.csv to pokemon.csv


In [6]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['pokemon.csv']))
df.head()


,id,user_id,QoA_VLCresolution,QoA_VLCbitrate,QoA_VLCframerate,QoA_VLCdropped,QoA_VLCaudiorate,QoA_VLCaudioloss,QoA_BUFFERINGcount,QoA_BUFFERINGtime,...,QoD_os-version,QoD_api-level,QoU_sex,QoU_age,QoU_Ustedy,QoF_begin,QoF_shift,QoF_audio,QoF_video,MOS
0,590,71,360,528.39294,24.950000,0,43.800000,0,2,683,...,4.1.1(122573.16),16,1,20,5,3,5,3,4,3
1,428,46,360,402.64664,30.066667,0,44.200000,0,2,690,...,4.4.2(I9195XXUCNK1),19,1,25,5,4,5,5,5,5
2,406,44,360,735.84070,24.200000,0,44.183333,0,2,840,...,4.1.2(I9300XXELL4),16,1,22,5,3,5,4,4,4
3,1256,133,240,529.48830,24.116667,9,43.850000,0,2,868,...,4.4.4(suv3Rw),19,1,31,5,4,5,5,5,5
4,244,22,360,736.00085,24.066667,0,43.850000,0,2,869,...,4.1.2(I9300XXELL4),16,0,26,5,5,5,4,5,5


In [8]:
df = df[[
    "QoA_VLCbitrate", "QoA_VLCframerate", "QoA_VLCdropped", "MOS"
]].copy()

df.rename(columns={
    "QoA_VLCbitrate": "bitrate",
    "QoA_VLCframerate": "framerate",
    "QoA_VLCdropped": "DroppedFrames"
}, inplace=True)

df["DroppedFrames"] = df["DroppedFrames"].astype(float)


In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [11]:
def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        sentiment = torch.argmax(probs) + 1
    return sentiment.item()


In [41]:
def mos_to_class(mos):
    if mos <= 2:
        return 0
    elif mos == 3:
        return 1
    else:
        return 2

df["MOS_class"] = df["MOS"].apply(mos_to_class)

print(df["MOS_class"].value_counts())



MOS_class
2    1086
1     246
0     211
Name: count, dtype: int64


In [46]:
labels = df["MOS_class"]
X = df.drop(columns=["MOS", "MOS_class"])



X_train, X_test, y_train, y_test = train_test_split(features.values, labels.values, test_size=0.2, random_state=42)

In [47]:
print("Unique labels in y_train:", set(y_train))

Unique labels in y_train: {np.int64(0), np.int64(1), np.int64(2)}


In [49]:
class QoSDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(QoSDataset(X_train, y_train), batch_size=16, shuffle=True)
test_loader = DataLoader(QoSDataset(X_test, y_test), batch_size=16)


In [50]:
class QoSNet(nn.Module):
    def __init__(self):
        super(QoSNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3)  \
        )
    def forward(self, x):
        return self.model(x)

model = QoSNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()


In [51]:
for epoch in range(20):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 383.9857
Epoch 2, Loss: 106.1320
Epoch 3, Loss: 91.0280
Epoch 4, Loss: 68.5141
Epoch 5, Loss: 57.3995
Epoch 6, Loss: 60.0948
Epoch 7, Loss: 83.8659
Epoch 8, Loss: 42.6190
Epoch 9, Loss: 36.4181
Epoch 10, Loss: 29.6667
Epoch 11, Loss: 34.4861
Epoch 12, Loss: 41.5788
Epoch 13, Loss: 21.5666
Epoch 14, Loss: 20.8205
Epoch 15, Loss: 20.8917
Epoch 16, Loss: 17.7103
Epoch 17, Loss: 15.7537
Epoch 18, Loss: 19.1795
Epoch 19, Loss: 19.7900
Epoch 20, Loss: 11.8590


In [52]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch)
        predicted = torch.argmax(preds, dim=1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 97.73%


In [54]:
def qos_action(row):
    X_input = torch.tensor([[
        row["bitrate"],
        row["framerate"],
        row["DroppedFrames"],
        row["sentiment"]
    ]], dtype=torch.float32)

    with torch.no_grad():
        pred = model(X_input)
        label = torch.argmax(pred, dim=1).item()

    if label == 0:
        return "Reduce resolution or switch server"
    elif label == 1:
        return "Adjust bitrate or buffer"
    else:
        return "Maintain current QoS"

df["qos_action"] = df.apply(qos_action, axis=1)
df[["sentiment", "MOS", "qos_action"]].head(10)


,sentiment,MOS,qos_action
0,3,3,Adjust bitrate or buffer
1,5,5,Maintain current QoS
2,5,4,Maintain current QoS
3,5,5,Maintain current QoS
4,5,5,Maintain current QoS
5,5,4,Maintain current QoS
6,5,5,Maintain current QoS
7,5,4,Maintain current QoS
8,3,3,Adjust bitrate or buffer
9,5,4,Maintain current QoS


In [57]:
!pip install transformers --quiet


In [59]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [60]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [62]:
def predict_sentiment(text):
  inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
  with torch.no_grad():
    outputs = sentiment_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    sentiment = torch.argmax(probs, dim=1).item() + 1
  return sentiment

In [64]:
if 'comment' in df.columns:
  df['sentiment'] = df['comment'].astype(str).apply(predict_sentiment)
else:
  print("Error: 'comment' column not found in DataFrame.")

Error: 'comment' column not found in DataFrame.


In [65]:
def qos_action(row):
  X_input = torch.tensor([[
  row["bitrate"],
  row["framerate"],
  row["DroppedFrames"],
  row["sentiment"]
  ]], dtype=torch.float32)

In [66]:
df["qos_action"] = df.apply(qos_action, axis=1)


In [67]:
print(df[["sentiment", "MOS", "qos_action"]].head(10))


   sentiment  MOS qos_action
0          3    3       None
1          5    5       None
2          5    4       None
3          5    5       None
4          5    5       None
5          5    4       None
6          5    5       None
7          5    4       None
8          3    3       None
9          5    4       None
